In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

In [19]:
xtrain = np.genfromtxt('X_train.csv', delimiter=',', skip_header=1)
ytrain = np.genfromtxt('y_train.csv', delimiter=',', skip_header=1)

xtrain = xtrain[:, 1:]
ytrain = ytrain[:, 1:]
nb_train = xtrain.shape[0] #number of data points in xtrain

xtrain = pd.DataFrame(xtrain)
xtrain = xtrain.fillna(xtrain.median())
xtrain = np.array(xtrain)

In [20]:
#random shuffling of data 

perm = np.random.permutation(xtrain.shape[0])
xtrain = xtrain[perm, :]
ytrain = xtrain[perm, :]

In [21]:
#splitting data for CV 

nb_folds = 10
parts_x = np.array_split(xtrain, nb_folds, axis=0)
parts_y = np.array_split(ytrain, nb_folds, axis=0)
 

In [22]:
avgr2_train = 0
avgr2_val = 0
for cv_iter in range(0, nb_folds):
    curr_train_x = np.vstack([p for i,p in enumerate(parts_x) if i != cv_iter])
    curr_train_y = np.vstack([p for i,p in enumerate(parts_y) if i != cv_iter]).ravel()
 
    curr_val_x = parts_x[cv_iter]
    curr_val_y = parts_y[cv_iter].ravel()
 
    model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
 
    model.fit(curr_train_x, curr_train_y.ravel())
    train_prediction = model.predict(curr_train_x)
    r2_train = r2_score(train_prediction, curr_train_y)
    avgr2_train += r2_train
 
    val_prediction = model.predict(curr_val_x)
    r2_val = r2_score(val_prediction, curr_val_y)
    avgr2_val += r2_val
    print('[Fold {:2d}] train: {:5.2f} val:{:5.2f}'.format(cv_iter, r2_train, r2_val))
   
avgr2_train /= nb_folds
avgr2_val /= nb_folds
print('[Average] train: {:5.2f} val: {:5.2f}'.format(avgr2_train, avgr2_val))


XGBoostError: [00:52:39] C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (1090 vs. 906880) : labels are not correctly providedpreds.size=1090, label.size=906880

In [90]:
######### RUN FROM HERE   ##########

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn import preprocessing
import xgboost as xgb

In [79]:
# Step 0: Load the proper csvs

xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

In [80]:
# Step 1: Treat missing values
# Treat missing values as column medians. Important (apparently) to use the medians from the training set in the test set

xtrain = xtrain.fillna(xtrain.median())
xtest = xtest.fillna(xtrain.median())

In [81]:
# Step 2: Scale the training and test data

scaler = preprocessing.StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)
xtest_scaled = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

In [82]:
# Step 3: Remove unnecessary id column from ytrain that just ***** things up

ytrain1 = ytrain.loc[:, "y"]
ytrain2 = pd.DataFrame(data = ytrain1.values, columns= ['y'])
ytrain = ytrain2

In [83]:
selected_feat = xtrain.columns[sel.get_support()]
xtrain = xtrain.loc[:,selected_feat]
xtest = xtest.loc[:,selected_feat]

In [84]:
'''

# Do weird Andreas PCA thing

from sklearn import decomposition
import matplotlib.pyplot as plt
pca = decomposition.PCA(n_components=2)
principal_components = pca.fit_transform(xtrain)
xpca = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])
# Do weird Andreas PCA thing

from sklearn import decomposition
import matplotlib.pyplot as plt
pca = decomposition.PCA(n_components=2)
principal_components = pca.fit_transform(xtrain)
xpca = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])

outliers = (xpca['PC1'] <= 9) & (xpca['PC1'] >= -8) & (xpca['PC2'] <= 7) & (xpca['PC2'] >= -7)
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]
'''

# Outlier detection with local outlier factor
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=700, contamination=0.1)
outliers = clf.fit_predict(xtrain)

# Remove outliers from xtrain and ytrain


outliers = outliers == 1
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]


1090


In [85]:
'''
# Step 4: Select top K columns using LassoCV

# Fit Lasso model on the data
from sklearn import linear_model
#reg = linear_model.LassoCV(n_alphas=10)
reg = linear_model.Lasso(alpha=0.3)
reg.fit(xtrain, ytrain)  
#reg.predict(xtest)

# Select top K features from Lasso
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(reg, prefit=True)
xtrain = model.transform(xtrain)

features = (reg.coef_ != 0)
#xtest_lasso = model.transform(xtest)
xtest = xtest.loc[:, features]
print((xtrain.shape, xtrain.shape))



from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
xtrain = poly.fit_transform(xtrain)

print(xtrain.shape)

# Create polynomial features for xtest
poly = PolynomialFeatures(2)
xtest = poly.fit_transform(xtest)
print(xtest.shape)

xtrain = pd.DataFrame(xtrain)
'''

'\n# Step 4: Select top K columns using LassoCV\n\n# Fit Lasso model on the data\nfrom sklearn import linear_model\n#reg = linear_model.LassoCV(n_alphas=10)\nreg = linear_model.Lasso(alpha=0.3)\nreg.fit(xtrain, ytrain)  \n#reg.predict(xtest)\n\n# Select top K features from Lasso\nfrom sklearn.feature_selection import SelectFromModel\nmodel = SelectFromModel(reg, prefit=True)\nxtrain = model.transform(xtrain)\n\nfeatures = (reg.coef_ != 0)\n#xtest_lasso = model.transform(xtest)\nxtest = xtest.loc[:, features]\nprint((xtrain.shape, xtrain.shape))\n\n\n\nfrom sklearn.preprocessing import PolynomialFeatures\npoly = PolynomialFeatures(2)\nxtrain = poly.fit_transform(xtrain)\n\nprint(xtrain.shape)\n\n# Create polynomial features for xtest\npoly = PolynomialFeatures(2)\nxtest = poly.fit_transform(xtest)\nprint(xtest.shape)\n\nxtrain = pd.DataFrame(xtrain)\n'

In [89]:
# k-fold cross validation evaluation of xgboost model

from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold


clf_xgb = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'r2', error_score = 0, verbose = 3, n_jobs = -1)


k = 10

cv_means = []
cv_stds = []
#model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
scores = cross_val_score(estimator = clf,
                             X = xtrain,
                             y = ytrain,
                             scoring = 'r2',
                             cv = k)
print(np.mean(scores))
#cv_means.append(np.mean(scores))
#cv_stds.append(np.std(scores))

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.5min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.6min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


...

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.8min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.3s


KeyboardInterrupt: 

In [ ]:
# k-fold cross validation evaluation of xgboost model

from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold


clf_xgb = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
param_dist = {'n_estimators': stats.randint(100, 300),
              'learning_rate': stats.uniform(0.01, 0.05),
              'subsample': stats.uniform(0.3, 0.),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'r2', error_score = 0, verbose = 3, n_jobs = -1)


k = 10

cv_means = []
cv_stds = []
#model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
scores = cross_val_score(estimator = clf,
                             X = xtrain,
                             y = ytrain,
                             scoring = 'r2',
                             cv = k)
print(np.mean(scores))
#cv_means.append(np.mean(scores))
#cv_stds.append(np.std(scores))

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.5min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.6min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.5min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  2.8min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.3s


In [75]:
model.fit(xtrain,ytrain)
predict = model.predict(xtest)
index = pd.read_csv("sample.csv")
index['y'] = predict

index.to_csv("trial.csv")

Need to try some fetures selection and outlier detection shit



In [35]:
np.mean(scores)

0.5175284544897842